In [1]:
import torch
import torch.nn as nn

# Define LSTM model
class LSTM(nn.Module):  # 继承nn.Module类，定义自己的LSTM类

    # Constructor
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()  # 调用nn.Module的构造函数
        self.hidden_size = hidden_size  # 设定隐藏状态的尺寸
        self.num_layers = num_layers  # 设定LSTM层数
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)  # 定义LSTM层，batch_first=True表示输入的第一个维度是batch_size
        self.fc = nn.Linear(hidden_size, output_size)  # 定义全连接层，将LSTM层的输出转换为最终的输出

    # Forward function
    def forward(self, x, h0, c0):
        # x: input of shape (batch_size, seq_len, input_size)
        # h0: initial hidden state of shape (num_layers, batch_size, hidden_size)
        # c0: initial cell state of shape (num_layers, batch_size, hidden_size)

        out, (hn, cn) = self.lstm(x, (h0, c0))  # 调用LSTM层进行前向运算，out为输出，(hn, cn)为最后一个时间步长的隐藏状态和单元状态
        out = self.fc(out[:, -1, :])  # 取最后一个时间步长的输出，并通过全连接层得到最终的输出结果
        # out: output of shape (batch_size, output_size)
        return out

# Test the model using example data
input_size, hidden_size, num_layers, output_size = 3, 4, 2, 2  # 设定模型参数
batch_size, seq_len = 5, 3
x = torch.randn(batch_size, seq_len, input_size)  # 生成随机输入张量
h0 = torch.randn(num_layers, batch_size, hidden_size)  # 生成随机初始隐藏状态
c0 = torch.randn(num_layers, batch_size, hidden_size)  # 生成随机初始单元状态
model = LSTM(input_size, hidden_size, num_layers, output_size)  # 实例化模型
output = model(x, h0, c0)  # 测试模型
print(output)  # 输出模型结果

tensor([[0.3831, 0.4444],
        [0.4287, 0.4069],
        [0.3927, 0.4125],
        [0.4042, 0.4212],
        [0.4081, 0.4554]], grad_fn=<AddmmBackward0>)


In [ ]:
class MyModel(torch.nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size):
        super(MyModel, self).__init__()
        
        self.hidden_size = hidden_size
        self.lstm = torch.nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size)
        c0 = torch.zeros(1, x.size(0), self.hidden_size)
        lstm_out, _ = self.lstm(x, (h0, c0))
        out = self.fc(lstm_out[:, -1, :])
        return out


In [ ]:
# Define your model
model = MyModel(input_size=7, hidden_size=16, output_size=1)

# Define loss function
criterion = torch.nn.MSELoss()

# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(1000):
    # Forward pass
    y_pred = model(x_train)

    # Compute loss
    loss = criterion(y_pred, y_train)

    # Zero gradients, backward pass, and update weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress
    if epoch % 100 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 1000, loss.item()))


In [ ]:
# Load your testing data
x_test = torch.tensor(x_test, dtype=torch.float)
y_test = torch.tensor(y_test, dtype=torch.float)

# Run the test data through the trained model to predict y_test_hat
with torch.no_grad():
    y_test_hat = model(x_test)

# Calculate mean squared error between y_test_hat and y_test
mse = torch.nn.functional.mse_loss(y_test_hat, y_test)

print("Mean Squared Error of the model on the test set: ", mse.item())

In [ ]:
import matplotlib.pyplot as plt

# Convert tensors to numpy arrays for plotting
x_test_np = x_test.numpy()
y_test_np = y_test.numpy()
y_test_hat_np = y_test_hat.numpy()

# Plot the testing data and predicted values
plt.plot(y_test_np, label='True')
plt.plot(y_test_hat_np, label='Predicted')

# Format the plot
plt.title('Testing Data vs Predicted Values')
plt.xlabel('Sample')
plt.ylabel('Value')
plt.legend()

# Display the plot
plt.show()